Source
https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py

In [16]:
from __future__ import print_function, division

import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LSTM
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import sys
import numpy as np
from utils import show_model 

import data_functions

In [17]:
#Load data from already created pickle file
words, X, dataX, dataY, n_words, n_vocab, index2word, word2index = data_functions.load_from_pickle("data.pickle")

In [18]:
from keras.utils import np_utils
# one hot encode the output variable

X100 = dataX[0:100]
trainX = np_utils.to_categorical(X100)
n_vocab = (trainX.shape[2])

# trainX.shape
# print(trainX[0])
# print(trainX.shape)
# print(X100[0])
# print(n_vocab)

In [19]:
class GAN():
    def __init__(self):
       
        self.input_shape = 100
        self.noise_dim = 50
        self.img_shape = (self.input_shape, 1)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.noise_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (self.noise_dim,)
        
#         #LSTM generator
#         model = LSTM(self.noise_dim, return_sequences=True, return_state=False, name='LSTM_generator')
#         outputs = model(embded_out, initial_state=encoder_states)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='sigmoid'))
        model.add(Reshape(self.img_shape))

        print(model.summary())
#         show_model(model, 20, 20)

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = self.img_shape
        print(self.img_shape)
        hidden_size = 100
        
        ################# LSTM disc ##############

        #LSTM discriminator
        disc_inputs = Input(shape=(img_shape), name='disc_inputs') 

        disc_lstm = LSTM(hidden_size, return_state=True, name='disc_LSTM')
        _, disc_state_h, disc_state_c = disc_lstm(disc_inputs)  

        disc_dense1 = Dense(50, activation='relu', name='disc_dense1')
        disc_dense1_out = disc_dense1(disc_state_h)

        disc_dense2 = Dense(1, activation='sigmoid', name='disc_dense2')
        disc_out = disc_dense2(disc_dense1_out)

        model = Model(disc_inputs, disc_out)

        print(model.summary())
#         show_model(model, 10, 10)  
        
        img = Input(shape=(img_shape))
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, save_interval=50):

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
#             print('imgs shape=',imgs.shape)

            noise = np.random.normal(0, 1, (half_batch, self.noise_dim))
#             print('noise shape=',noise.shape)
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            if epoch % 10 == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r = 5
        noise = np.random.normal(0, 1, (r , self.noise_dim))
        gen_imgs = self.generator.predict(noise)
        pred = np.reshape(gen_imgs * float(n_vocab), (r,-1))
        pred[pred < 0] = 0
        pred[pred >= n_vocab-1] = 0
        pred = pred.astype(int)        
        for p in pred:
            s = [index2word[i+1] for i in p]
            s = ' '.join(s)
            print(s)
            print('----------------------')

gan = GAN()

(100, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
disc_inputs (InputLayer)     (None, 100, 1)            0         
_________________________________________________________________
disc_LSTM (LSTM)             [(None, 100), (None, 100) 40800     
_________________________________________________________________
disc_dense1 (Dense)          (None, 50)                5050      
_________________________________________________________________
disc_dense2 (Dense)          (None, 1)                 51        
Total params: 45,901
Trainable params: 45,901
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 256)               13056     
__________________________________________________________

In [20]:
gan.train(X[0:200], epochs=30000, batch_size=32, save_interval=200)

0 [D loss: 0.692809, acc.: 50.00%] [G loss: 0.698909]
فانظروا ومنافع داعي وسبحان ليهلك تطلع فإنك هداكم ثياب هامان المريض قتلت كلم وهب الغني عنا الوسيلة أوحينا أحسنوا تنزل بذلك متعمدا ودا خاطئين فسادا حديث ثاني ودوا اضرب كالأنعام فألقى مقام كفورا وأنفقوا لفرعون ولعلكم بالحكمة فبئس شططا حبة الطاغوت الأمثال لعنت ملكت لأظنه تأس أولادكم القهار أبلغ رأوه الموج أموال يحض وأطعموا مال أنبئكم جنوبهم مذموما مقتدر كيدهم وجيها لقوم وظلموا وأقام مرض ظالمين والنخل الشهادة المشرقين تمسوهن سوءة صراطا والإنس أبي السمع ذكروا بالنفس بعدت لطيف تلبسونها برهانكم عبدنا جاءوك وجعل أيديكم ومنذرين يعمل فاصبروا فاستوى أجمعون تمور الذكرى أغرقنا نسيتم وحفظا فأخرج فاصبروا يحكم فأغرقناهم وآمنوا
----------------------
عاليها رسولهم يعيدكم سرر متين ببعض الذنوب ملكه عاكفون وأطعنا بخلوا بناء يشترون آلهتهم يغنوا بالحسنى تزكى الرجفة وأهلها تتبع باغ يتم أولي شهدنا قصورا وصدق فدية اللطيف بارئكم صدورهم الجبال لآبائهم خرجوا القرنين أحصنت ويهديهم وأنهارا عرش يأمر حجابا لمؤمن فجعلناهم اذهب قرن سليمان مذموما تكونوا تذكروا حمل الو

410 [D loss: 0.492596, acc.: 75.00%] [G loss: 1.872748]
420 [D loss: 0.518632, acc.: 71.88%] [G loss: 1.909292]
430 [D loss: 0.574908, acc.: 65.62%] [G loss: 1.807360]
440 [D loss: 0.543192, acc.: 68.75%] [G loss: 1.652367]
450 [D loss: 0.600302, acc.: 62.50%] [G loss: 1.938210]
460 [D loss: 0.598485, acc.: 62.50%] [G loss: 1.864420]
470 [D loss: 0.570768, acc.: 65.62%] [G loss: 1.905192]
480 [D loss: 0.571670, acc.: 65.62%] [G loss: 2.195895]
490 [D loss: 0.541442, acc.: 68.75%] [G loss: 1.851784]
500 [D loss: 0.570308, acc.: 65.62%] [G loss: 1.877715]
510 [D loss: 0.516574, acc.: 71.88%] [G loss: 2.031692]
520 [D loss: 0.568616, acc.: 65.62%] [G loss: 1.785030]
530 [D loss: 0.540359, acc.: 68.75%] [G loss: 2.072767]
540 [D loss: 0.596368, acc.: 62.50%] [G loss: 1.821054]
550 [D loss: 0.568813, acc.: 65.62%] [G loss: 1.848143]
560 [D loss: 0.619668, acc.: 59.38%] [G loss: 2.731866]
570 [D loss: 0.619426, acc.: 59.38%] [G loss: 2.170127]
580 [D loss: 0.515660, acc.: 71.88%] [G loss: 1.

1010 [D loss: 0.598255, acc.: 62.50%] [G loss: 1.544562]
1020 [D loss: 0.569671, acc.: 65.62%] [G loss: 1.685583]
1030 [D loss: 0.568338, acc.: 65.62%] [G loss: 1.704724]
1040 [D loss: 0.597947, acc.: 62.50%] [G loss: 1.861060]
1050 [D loss: 0.569527, acc.: 65.62%] [G loss: 1.747954]
1060 [D loss: 0.673175, acc.: 53.12%] [G loss: 1.774642]
1070 [D loss: 0.624070, acc.: 59.38%] [G loss: 2.203510]
1080 [D loss: 0.619894, acc.: 59.38%] [G loss: 1.812708]
1090 [D loss: 0.569915, acc.: 65.62%] [G loss: 1.691864]
1100 [D loss: 0.597922, acc.: 62.50%] [G loss: 1.844186]
1110 [D loss: 0.546358, acc.: 68.75%] [G loss: 2.004935]
1120 [D loss: 0.597990, acc.: 62.50%] [G loss: 1.435996]
1130 [D loss: 0.571742, acc.: 65.62%] [G loss: 1.877883]
1140 [D loss: 0.630625, acc.: 59.38%] [G loss: 2.041269]
1150 [D loss: 0.548202, acc.: 68.75%] [G loss: 2.056901]
1160 [D loss: 0.549139, acc.: 68.75%] [G loss: 1.616995]
1170 [D loss: 0.600710, acc.: 62.50%] [G loss: 1.782871]
1180 [D loss: 0.520052, acc.: 7

1610 [D loss: 0.600306, acc.: 62.50%] [G loss: 3.096537]
1620 [D loss: 0.707052, acc.: 50.00%] [G loss: 2.327960]
1630 [D loss: 0.572994, acc.: 65.62%] [G loss: 2.332284]
1640 [D loss: 0.625516, acc.: 59.38%] [G loss: 2.337321]
1650 [D loss: 0.599258, acc.: 62.50%] [G loss: 1.749089]
1660 [D loss: 0.626770, acc.: 59.38%] [G loss: 2.341770]
1670 [D loss: 0.651586, acc.: 56.25%] [G loss: 1.755113]
1680 [D loss: 0.597868, acc.: 62.50%] [G loss: 2.764606]
1690 [D loss: 0.546011, acc.: 68.75%] [G loss: 2.162464]
1700 [D loss: 0.572840, acc.: 65.62%] [G loss: 2.166926]
1710 [D loss: 0.680705, acc.: 53.12%] [G loss: 2.375885]
1720 [D loss: 0.679703, acc.: 53.12%] [G loss: 2.389793]
1730 [D loss: 0.572877, acc.: 65.62%] [G loss: 2.599612]
1740 [D loss: 0.573283, acc.: 65.62%] [G loss: 2.396098]
1750 [D loss: 0.545121, acc.: 68.75%] [G loss: 2.397308]
1760 [D loss: 0.600080, acc.: 62.50%] [G loss: 1.996452]
1770 [D loss: 0.544718, acc.: 68.75%] [G loss: 2.617301]
1780 [D loss: 0.653482, acc.: 5

2210 [D loss: 0.598668, acc.: 62.50%] [G loss: 2.768045]
2220 [D loss: 0.571747, acc.: 65.62%] [G loss: 2.334599]
2230 [D loss: 0.545621, acc.: 68.75%] [G loss: 2.337687]
2240 [D loss: 0.624308, acc.: 59.38%] [G loss: 3.456282]
2250 [D loss: 0.546297, acc.: 68.75%] [G loss: 2.573476]
2260 [D loss: 0.571663, acc.: 65.62%] [G loss: 2.808432]
2270 [D loss: 0.571697, acc.: 65.62%] [G loss: 1.904625]
2280 [D loss: 0.517025, acc.: 71.88%] [G loss: 3.042336]
2290 [D loss: 0.546998, acc.: 68.75%] [G loss: 2.593974]
2300 [D loss: 0.623877, acc.: 59.38%] [G loss: 2.602023]
2310 [D loss: 0.625575, acc.: 59.38%] [G loss: 2.604301]
2320 [D loss: 0.653651, acc.: 56.25%] [G loss: 1.935579]
2330 [D loss: 0.624093, acc.: 59.38%] [G loss: 2.624497]
2340 [D loss: 0.571356, acc.: 65.62%] [G loss: 1.935465]
2350 [D loss: 0.628492, acc.: 59.38%] [G loss: 2.858696]
2360 [D loss: 0.622895, acc.: 59.38%] [G loss: 2.404716]
2370 [D loss: 0.600582, acc.: 62.50%] [G loss: 2.176892]
2380 [D loss: 0.626962, acc.: 5

...

10210 [D loss: 0.714175, acc.: 53.12%] [G loss: 0.820560]
10220 [D loss: 0.617735, acc.: 68.75%] [G loss: 0.860658]
10230 [D loss: 0.641974, acc.: 65.62%] [G loss: 0.782387]
10240 [D loss: 0.641611, acc.: 62.50%] [G loss: 0.829314]
10250 [D loss: 0.621601, acc.: 78.12%] [G loss: 0.798182]
10260 [D loss: 0.700338, acc.: 50.00%] [G loss: 0.910181]
10270 [D loss: 0.665777, acc.: 81.25%] [G loss: 0.827629]
10280 [D loss: 0.676107, acc.: 59.38%] [G loss: 0.813156]
10290 [D loss: 0.687128, acc.: 56.25%] [G loss: 0.827648]
10300 [D loss: 0.640589, acc.: 65.62%] [G loss: 0.828391]
10310 [D loss: 0.663134, acc.: 59.38%] [G loss: 0.811416]
10320 [D loss: 0.684709, acc.: 56.25%] [G loss: 0.849138]
10330 [D loss: 0.696875, acc.: 50.00%] [G loss: 0.846831]
10340 [D loss: 0.595785, acc.: 71.88%] [G loss: 0.828029]
10350 [D loss: 0.711339, acc.: 53.12%] [G loss: 0.845188]
10360 [D loss: 0.688814, acc.: 65.62%] [G loss: 0.815055]
10370 [D loss: 0.681573, acc.: 53.12%] [G loss: 0.873956]
10380 [D loss:

10810 [D loss: 0.667427, acc.: 65.62%] [G loss: 0.629297]
10820 [D loss: 0.710431, acc.: 50.00%] [G loss: 0.693010]
10830 [D loss: 0.619373, acc.: 78.12%] [G loss: 0.738860]
10840 [D loss: 0.668001, acc.: 62.50%] [G loss: 0.713957]
10850 [D loss: 0.612969, acc.: 68.75%] [G loss: 0.789907]
10860 [D loss: 0.680153, acc.: 56.25%] [G loss: 0.819259]
10870 [D loss: 0.634701, acc.: 65.62%] [G loss: 0.773563]
10880 [D loss: 0.664062, acc.: 59.38%] [G loss: 0.914594]
10890 [D loss: 0.594293, acc.: 65.62%] [G loss: 0.940349]
10900 [D loss: 0.584885, acc.: 65.62%] [G loss: 0.988899]
10910 [D loss: 0.639065, acc.: 59.38%] [G loss: 1.045442]
10920 [D loss: 0.624133, acc.: 59.38%] [G loss: 1.035967]
10930 [D loss: 0.514226, acc.: 78.12%] [G loss: 1.006765]
10940 [D loss: 0.645732, acc.: 62.50%] [G loss: 1.118818]
10950 [D loss: 0.547885, acc.: 71.88%] [G loss: 1.313703]
10960 [D loss: 0.581558, acc.: 65.62%] [G loss: 1.277441]
10970 [D loss: 0.629065, acc.: 65.62%] [G loss: 1.171393]
10980 [D loss:

11410 [D loss: 0.585144, acc.: 71.88%] [G loss: 1.060367]
11420 [D loss: 0.641246, acc.: 62.50%] [G loss: 1.098035]
11430 [D loss: 0.612968, acc.: 71.88%] [G loss: 1.012111]
11440 [D loss: 0.596776, acc.: 68.75%] [G loss: 1.090877]
11450 [D loss: 0.586778, acc.: 65.62%] [G loss: 0.959865]
11460 [D loss: 0.581975, acc.: 65.62%] [G loss: 1.089914]
11470 [D loss: 0.608508, acc.: 65.62%] [G loss: 1.219584]
11480 [D loss: 0.540159, acc.: 68.75%] [G loss: 0.944154]
11490 [D loss: 0.767784, acc.: 46.88%] [G loss: 1.099549]
11500 [D loss: 0.686166, acc.: 59.38%] [G loss: 0.669500]
11510 [D loss: 0.739770, acc.: 40.62%] [G loss: 0.688506]
11520 [D loss: 0.685771, acc.: 62.50%] [G loss: 0.726211]
11530 [D loss: 0.693692, acc.: 56.25%] [G loss: 0.717944]
11540 [D loss: 0.689081, acc.: 65.62%] [G loss: 0.723785]
11550 [D loss: 0.705002, acc.: 50.00%] [G loss: 0.733201]
11560 [D loss: 0.669855, acc.: 68.75%] [G loss: 0.696808]
11570 [D loss: 0.685282, acc.: 50.00%] [G loss: 0.753035]
11580 [D loss:

12010 [D loss: 0.645395, acc.: 59.38%] [G loss: 0.984980]
12020 [D loss: 0.666522, acc.: 59.38%] [G loss: 0.960841]
12030 [D loss: 0.601473, acc.: 62.50%] [G loss: 0.985932]
12040 [D loss: 0.603328, acc.: 71.88%] [G loss: 1.035210]
12050 [D loss: 0.721671, acc.: 50.00%] [G loss: 0.587971]
12060 [D loss: 0.647685, acc.: 65.62%] [G loss: 0.729321]
12070 [D loss: 0.636507, acc.: 56.25%] [G loss: 0.815712]
12080 [D loss: 0.513022, acc.: 68.75%] [G loss: 1.022975]
12090 [D loss: 0.658348, acc.: 31.25%] [G loss: 0.922379]
12100 [D loss: 0.610852, acc.: 62.50%] [G loss: 1.141006]
12110 [D loss: 0.673531, acc.: 56.25%] [G loss: 0.939046]
12120 [D loss: 0.618715, acc.: 59.38%] [G loss: 0.917479]
12130 [D loss: 0.597846, acc.: 62.50%] [G loss: 1.115445]
12140 [D loss: 0.652311, acc.: 56.25%] [G loss: 1.130090]
12150 [D loss: 0.617681, acc.: 62.50%] [G loss: 1.092494]
12160 [D loss: 0.694727, acc.: 40.62%] [G loss: 0.678936]
12170 [D loss: 0.660434, acc.: 59.38%] [G loss: 0.758748]
12180 [D loss:

KeyboardInterrupt: 